In [1]:
import pandas as pd 
import numpy as np
import os
import win32com.client
pd.options.display.max_columns = None 

In [14]:
class Prisma():
    
    def __init__(self,filepath):
        """
        Inputs: filepath of media plan
        """
        try:
            self.df = pd.read_csv(filepath)
        except:
            self.df= pd.read_excel(filepath)
        
    def clean(self):
        """
        OBJ: Cleans Prisma Media Plant to Placement Level of Granularity
        Outputs: DataFrame but not explicit
        """
        
        #create package column
        
        #If statement of where Supplier Column is NOT empty, then fill Package Column with value from Name Column 
        self.df['Package Name'] = np.where(self.df['Supplier'].isnull(), np.nan, self.df['Name'])
        #Forward fill Package Name column
        self.df['Package Name'] = self.df['Package Name'].fillna(method='ffill')

        #If statement of where Positioning Column is empty, then fill Placement name column with value from Name Column
        self.df['Placement Name'] = np.where(self.df['Positioning'].isnull(),np.nan, self.df['Name'])

        #Create a list of columns needed to be forward filled.
        ColsToFill = ['Supplier','Start date','End date','Cost method','Cost','Rate','Units']
        #Fill those said columns
        self.df[ColsToFill] = self.df[ColsToFill].fillna(method='ffill')

        #drops package rows
        self.df = self.df[self.df['Placement Name'].notnull()].copy()
    
        return self.df
    
    def SiteServe(self,StartDate,EndDate,dcmFileForColumns):
        
        self.StartDate = StartDate
        self.EndDate = EndDate 
        
        """
        OBJ: Subets cleaned media plan to site served placements and repeats placement at day level
        Inputs: Start date AND dates of required reporting for site serve, DCM file for creative columns
        Outputs
        """
        #Repeat Placements by day level
        
        #Find difference betweeen start and end date
        self.df['DayDelta'] = pd.to_datetime(self.df['End date'])-pd.to_datetime(self.df['Start date']) 
        self.df['DayDelta'] = self.df['DayDelta'].dt.days + 1
        
        #Repeat rows by difference of start and end date
        self.df = self.df.loc[self.df.index.repeat(self.df['DayDelta'])]
        
        # Creates a new column of 1 day increments starting from Start Date to End Date of placement
        self.df['Reporting Date'] =(self.df.groupby(level=0)['Start date'].transform(lambda x: pd.date_range(start=x.iloc[0], periods=len(x))))
        self.df['Reporting Date'] = pd.to_datetime(self.df['Reporting Date'],format='%m-%d-%y')

        #Remove DayDelta
        self.df = self.df.drop('DayDelta',axis=1)
        
        #subset placement name for relevent columns
        self.df['ServingRule'] = self.df['Placement Name'].str.split('_',expand=True)[7]
        self.df['PrismaID'] = self.df['Placement Name'].str.split('_',expand=True)[0]
        
        #Create New dataframe based on Serving Rules
        self.SSdf = self.df[self.df['ServingRule'].isin(['SSNT','SSPT'])]
        #self.SSdf = self.df
        
        #Read in DCM FILE for creative names
        dcmColumns = pd.read_csv(dcmFileForColumns,skiprows=11)
        #dcmColumns = pd.read_excel(dcmFileForColumns)
        #Rename columns
        dcmColumns.rename(columns = {'Placement':'Placement Name',
                                    'Placement ID':'DCM ID'},inplace=True)
        #Create Prisma ID
        dcmColumns['PrismaID'] = dcmColumns['Placement Name'].str.split('_',expand=True)[0]
        
        #Full JOIN of DCM file and Prisma File
        self.SSdf = pd.merge(self.SSdf,dcmColumns[['PrismaID','Creative','Creative ID','DCM ID']],on='PrismaID', how='left')
        
        #Obtain relevant columns
        self.SSdf = self.SSdf[['Reporting Date','Campaign name','Supplier','DCM ID','Placement Name','Creative','Creative ID']] 
        #Subset by reporting date inputtted 
        self.SSdf = self.SSdf[(self.SSdf['Reporting Date'] >= pd.to_datetime(StartDate)) & (self.SSdf['Reporting Date'] <= pd.to_datetime(EndDate))]
        self.SSdf = self.SSdf.drop_duplicates()
        
        return self.SSdf
    
    def SiteServeTemplates(self):
        """
        OBJ: Creates Excel Templates of Sites served placements
        Outputs: Excel files
        """        
        
        #group by Suppler (could be Site also)
        DataFrameGroupedBySite = self.SSdf.groupby('Supplier')
        #creates a list of dataframes, where each dataframe is 
        Vendorlist= [contents for vendor, contents in DataFrameGroupedBySite]

        for vendor in Vendorlist:
            #Creating the export excelName; setting by the first Campaign name and Supplier value
            #campaignName = vendor['Campaign name'].iloc[0]
            supplierName = vendor['Supplier'].iloc[0]
            string = ('{}'+'.xlsx').format(supplierName)
            
            writer = pd.ExcelWriter(string, engine='xlsxwriter')
            vendor.to_excel(writer, sheet_name='SITE',index=False)
            workbook = writer.book
            worksheet = writer.sheets['SITE']

            #worksheet.protect()
            unlocked = workbook.add_format({'locked': 0})
            DateFormat = workbook.add_format({'num_format':'mm/dd/yyyy'})

            #text_format = workbook.add_format({'text_wrap': True})

            worksheet.write('H1','Spend')
            worksheet.write('I1','Delivered Impressions')
            worksheet.write('J1','Clicks')
            worksheet.write('K1','Video Impressions')
            worksheet.write('L1','Video: Starts')
            worksheet.write('M1','Video Completes')

            worksheet.set_column('F2:O2',25,unlocked) #where the vendors will input data
            worksheet.set_column('A:D',vendor['Campaign name'].map(len).max())
            worksheet.set_column('E:E',vendor['Placement Name'].map(len).max())
            worksheet.set_column('A:A',None,DateFormat)

            workbook.add_format({'text_wrap': True})

            workbook.close()
            writer.save()
            
    def email():
    
        contacts = pd.read_excel(r'C:\Users\hchaw\Desktop\EmailSheet.xlsx')

        for index, row in contacts.iterrows():
        
            subject = '{} Site Serve Template'.format(row['Company'])

            body = """Hi {},\n
Please find the attached document for site serve reporting for {} for the dates of X-X.

    •Reporting should be for every individual placement and creative that was live for this provided timeframe 
        oPlease ensure placement names and IDs and creative names and IDs match those provided in the template
    •Please ensure all columns are filled and there are no blanks or special characters in any of the cells. If you are unable to provide reporting for a certain metric, please let us know in your email response
    •For any zero values, please simply leave as “0”
    •In addition, please include any custom metrics in a separate tab. 

Please let us know if you will be able to deliver by X at EOD.

Thanks,
Harry""".format(row['First Name'],row['Company'],self.StartDate,self.EndDate)
        #outlook instance
        
            obj = win32com.client.Dispatch("Outlook.Application")
            #creates email
            newMail = obj.CreateItem(0x0)
            newMail.Subject = subject
            newMail.Body = body
            newMail.To = row['Email']
            #newMail.Attachments.Add(file)
            newMail.display()
            #newMail.Send()

        
def SS_Template(PrismaFile,StartDate,EndDate,CreativeColDcm,output):
        os.chdir(r'C:\Users\hchaw\Downloads')
        obj = Prisma(PrismaFile)
        obj.clean()
        obj.SiteServe(StartDate,EndDate,CreativeColDcm)
        os.chdir('C:/Users/hchaw/Desktop/Templates/'+output)
        obj.SiteServeTemplates()

### SS Tempaltes

In [4]:
#Kim Crawford
SS_Template(r'G:\Analytics\Constellation Wine\FY20\Data\KimCrawford\Media Plan\Archive\FY20_KimCrawford_MediaPlan.csv',
           '6/24/2019',
           '8/18/2019',
           r'C:\Users\hchaw\Downloads\1068_KimCrawzCreativeColumns_20190726_142707_2603687961.csv','Kim Craw')
#Woodbridge
SS_Template(r'G:\Analytics\Constellation Wine\FY20\Data\WoodBridge\Media Plan\Archive\Media Plan for CPRPSSWine_Woodbridge_8326015_FY20_Digital_Awareness_Nat 2019-07-10 (1).csv',
           '6/24/2019',
           '8/18/2019',
           r'C:\Users\hchaw\Downloads\1068_WoodbridgezCreativeColumns_20190726_143530_2603689619.csv','Woods')
#DreamingTreez
SS_Template(r'G:\Analytics\Constellation Wine\FY20\Data\DreamingTree\Media Plan\FY20_DreamingTree_MediaPlan.xlsx',
           '6/17/2019',
           '8/18/2019',
           r'C:\Users\hchaw\Downloads\1068_DreamingTreezCreativeColumns_20190726_145613_2603691726.csv','DreamingTree')

#Ruffino
SS_Template(r'G:\Analytics\Constellation Wine\FY20\Data\Ruffino\Media Plan\Ruffino_MediaPlan.xlsx',
           '7/1/2019',
           '8/4/2019',
           r'C:\Users\hchaw\Downloads\1068_RuffinozCreativeColumns_20190802_154702_2610597654.csv','Ruffino')
#Svedka
SS_Template(r'G:\Analytics\Constellation Wine\FY20\Data\Svedka\Media Plan\Media Plan for CPRRPFSpirits_SVEDKA_8288553_FY20_Digital_Awareness_Nati 2019-08-06.xlsx',
           '7/29/2019',
           '8/15/2019',
           r'C:\Users\hchaw\Downloads\1068_SvedkaCreativeColumns_20190819_203206_2629439556.csv','SVEDKA')

#Prisoner
SS_Template(r'C:\Users\hchaw\Downloads\Media Plan for CPT1RYWine_TPWC_8324404_FY20_Digital_Awareness_National_ 2019-08-23.csv',
           '8/14/2019',
           '9/8/2019',
           r'C:\Users\hchaw\Downloads\1068_SvedkaCreativeColumns_20190823_181632_2633742889.csv','Prisoner')

#RMPS
SS_Template(r'G:\Analytics\Constellation Wine\FY20\Data\RMPS\Digital\Media Plan\Media Plan for CPRPRVWine_RMPS_8294601_FY20_Digital_Awareness_National_ 2019-09-11.xlsx',
           '8/26/2019',
           '9/30/2019',
           r'C:\Users\hchaw\Downloads\1068_RMPSCreativeColumns_20190905_194913_2648235072.csv','RMPS')

#7Moons
SS_Template(r'G:\Analytics\Constellation Wine\FY20\Data\7 Moons\Media Plan\Media Plan for CPS50LWine_7 Moons_TBD_FY20_Digital_Awareness_National_A 2019-07-15.xlsx',
           '7/1/2019',
           '9/9/2019',
           r'C:\Users\hchaw\Downloads\1068_7MoonsCreativeColumns_20190809_193425_2618753598.csv','Moons')
#Woodbridge
SS_Template(r'G:\Analytics\Constellation Wine\FY20\Data\WoodBridge\Media Plan\Archive\Media Plan for CPRPSSWine_Woodbridge_8326015_FY20_Digital_Awareness_Nat 2019-07-10 (1).csv',
           '6/24/2019',
           '9/15/2019',
           r'C:\Users\hchaw\Downloads\1068_WoodbridgezCreativeColumns_20190726_143530_2603689619.csv','Woods')

In [186]:
#Kim Crawford
SS_Template(r'G:\Analytics\Constellation Wine\FY20\Data\KimCrawford\Media Plan\Archive\FY20_KimCrawford_MediaPlan.csv',
           '6/24/2019',
           '10/4/2019',
           r'C:\Users\hchaw\Downloads\1068_KimCrawzCreativeColumns_20190726_142707_2603687961.csv','Kim Craw')
#Svedka
SS_Template(r'G:\Analytics\Constellation Wine\FY20\Data\Svedka\Media Plan\Media Plan for CPRRPFSpirits_SVEDKA_8288553_FY20_Digital_Awareness_Nati 2019-08-06.xlsx',
           '9/30/2019',
           '10/13/2019',
           r'C:\Users\hchaw\Downloads\1068_SvedkaCreativeColumns_20190819_203206_2629439556.csv','SVEDKA')

PermissionError: [Errno 13] Permission denied: 'HULU.xlsx'

# Media Plan Check

In [ ]:
os.chdir(r'C:\Users\hchaw\Downloads')
test4 = Prisma('Media Plan for CPRXQXWine_Ruffino_TBD_FY20_Digital_Awareness_National_A 2019-06-17.csv')
test4.clean()
test4.df.head()
test4.df['Placement_Cost Method'] = test4.df['Placement Name'].str.split('_',expand=True)[6]
test4.df['Placement_Serving Rule'] = test4.df['Placement Name'].str.split('_',expand=True)[7]
test4.df['Placement_Start Date'] = test4.df['Placement Name'].str.split('_',expand=True)[14]

#test4.df[['Buy type','Positioning']].drop_duplicates()
test4.df['Placement_Display Type'] = test4.df['Placement Name'].str.split('_',expand =True)[5]
#test4.df[['Placement_Display Type','Positioning']].drop_duplicates()
test4.df[['Site','Placement_Cost Method','Cost method','Start date','Placement_Start Date','AD SERVER','Placement_Serving Rule',]].drop_duplicates()

# Convert CSV to Excel

In [21]:
import os
import glob
import csv
from xlsxwriter.workbook import Workbook

os.chdir(r'G:\Analytics\Constellation Wine\FY20\Data\Ruffino\Digital\Through 8.4\MOAT')

for csvfile in glob.glob(os.path.join('.', '*.csv')):
    workbook = Workbook(csvfile[:-4] + '.xlsx')
    worksheet = workbook.add_worksheet()
    with open(csvfile, 'rt', encoding='utf8') as f:
        reader = csv.reader(f)
        for r, row in enumerate(reader):
            for c, col in enumerate(row):
                worksheet.write(r, c, col)
    workbook.close()

# CPM Spend Calc

In [16]:
def cpm_spend(prisma,dcm):
    df= Prisma(prisma)
    df = df.clean()
    #df['Rate'] =df['Rate'].str.replace('$','')
    #df['Rate'] = df['Rate'].astype(float)
    rate = df[['Package Name','Rate']].drop_duplicates()

    dcm = pd.read_excel(dcm,sheet_name='Sheet1')
    var = pd.merge(dcm,df,left_on='Placement',right_on='Placement Name').groupby(['Supplier','Package Name','Placement']).sum().reset_index()
    var2 = pd.merge(rate,var,on='Package Name')
    #var2['cost'] = var2['Impressions']*var2['Rate_x']/1000
    
    return var2

In [55]:
cpm_spend(r'G:\Analytics\Constellation Wine\FY20\Data\WoodBridge\Media Plan\Archive\Media Plan for CPRPSSWine_Woodbridge_8326015_FY20_Digital_Awareness_Nat 2019-07-10 (1).csv',
          r'C:\Users\hchaw\Downloads\wood_DCM.xlsx').to_clipboard()

In [209]:
cpm_spend(r'G:\Analytics\Constellation Wine\FY20\Data\KimCrawford\Media Plan\Archive\Media Plan for CPRPSYWine_Kim Crawford_8322575_FY20_Digital_Awareness_N 2019-07-09 (2).csv',
          r'G:\Analytics\Constellation Wine\FY20\Data\KimCrawford\Reporting Through 8.19\Digital\KimCraw_DCM_6.24_8.18.xlsx').to_clipboard()

In [184]:
cpm_spend(r'G:\Analytics\Constellation Wine\FY20\Data\RMPS\Digital\Media Plan\Media Plan for CPRPRVWine_RMPS_8294601_FY20_Digital_Awareness_National_ 2019-09-11_V2.xlsx',
         r'C:\Users\hchaw\Downloads\RMPS_DCM.xlsx').to_clipboard()

In [83]:
cpm_spend(r'G:\Analytics\Constellation Wine\FY20\Data\Ruffino\Media Plan\Ruffino_MediaPlan_V3.xlsx',
         r'C:\Users\hchaw\Downloads\ruffino_DCM.xlsx').to_clipboard()

In [17]:
cpm_spend(r'G:\Analytics\Constellation Wine\FY20\Data\Meiomi\Media Plan\Meiomi_MediaPlan.xlsx',
         r'C:\Users\hchaw\Downloads\M_DCM.xlsx').to_clipboard()

# Search

In [187]:
class search():
    
    def __init__(self,file1,file2):
        self.ga = pd.read_excel(file1,sheet_name='Dataset1') 
        self.s360 = pd.read_excel(file2)
        
    def clean_ga(self):
        self.ga.rename(columns = {'Google Ads: Ad Group': 'Ad group',
                             'Date':'Start Date',
                              'Pages / Sessions': 'Pages per Visit'},inplace =True)
        self.ga['All Labels'] = self.ga['Campaign'].str.split(' - ',expand=True)[0]
        self.ga = self.ga[self.ga['Source / Medium'] == 'google / cpc'].copy()
        self.ga['Bounce Rate'] = self.ga['Bounces']/self.ga['Entrances']

    def clean_search(self):
        self.s360['All Labels'] = self.s360['Campaign'].str.split(' - ',expand=True)[0]
        self.s360.rename(columns={'From':'Start Date',
                             'To':'End Date'},inplace=True)
        self.s360['WAP'] =self.s360['Impr']/self.s360['Avg pos']
        
    def QA(self):
        self.df = pd.concat([self.ga,self.s360],sort=False)
        self.df =self.df[self.df['All Labels'] != '(not set)'].copy()
        return self.df
    
    def output(self,outputFilePath):
        df = pd.concat([self.ga,self.s360])
        columns = []
        df = df[columns].copy()
        #df.to_csv(outputFilePath)

In [ ]:
_ = search('G:\Analytics\Constellation Wine\FY20\Data\KimCrawford\Reporting Through 7.28\SEM\Analytics All Web Site Data KimCraw_FY20 20190617-20190728.xlsx',
          'G:\Analytics\Constellation Wine\FY20\Data\KimCrawford\Reporting Through 7.28\SEM\Kim Crawford - Google Search.xlsx')
_.clean_ga()
_.clean_search()
_.QA()

In [11]:
df = pd.read_excel(r'C:\Users\hchaw\Downloads\1068_RMPSCreativeColumns_20190905_194913_2648235072.xlsx',sheet_name='Sheet2')

In [ ]:
df[['Site (DCM)','Placement','Creative']].drop_duplicates().groupby(['Site (DCM)','Placement']).size().sort_values(ascending=False)

In [58]:
df = pd.read_excel(r'C:\Users\hchaw\Desktop\Monopoly Deal_DV.xlsx')

In [ ]:
df['Date_year'] =df['DayID'].astype(str).str[:4]

In [ ]:
df['DayID'].astype(str).str[:4],df['DayID'].astype(str).str[4:6],df['DayID'].astype(str).str[-2:],

# PGA Data cleaning

In [179]:
def PGA(file,StartDate,EndDate):
    timeDelta = (pd.to_datetime(startDate)-pd.to_datetime(endDate)).days +1 
    df = pd.read_excel(file)
    df['SS_Impressions'] = df['Gross Delivered Impressions']/timeDelta
    df['SS_Video Completions'] = df['Video Ads 100% Complete']/timeDelta
    df['SS_Video Views'] = df['Gross Delivered Impressions Div']
    df['SS_Video Starts'] = df['Gross Delivered Impressions Div']
    df['Publisher'] = 'PGA TOUR'
    df['Placement'] ='P13NQ20_0_PGA TOUR_Live Streaming Video_OLVSD_15_CPM_SSPT_DVMM_NAUD_CXL_GEN_PGA Contextual_CRPLAT_2019-09-16_2019-10-31_NA_1 x 1_1x1CC'
    
    df = pd.concat([df]*timeDelta,ignore_index=False)[['Placement',
                                                       'Creative Name',
                                                       'SS_Impressions',
                                                       'SS_Video Competions',
                                                       'SS_Video Views', 
                                                       'SS_Video Starts', 
                                                       'Publisher',
                                                       'Placement Name']]
    df.to_clipboard()

In [26]:
(pd.to_datetime('6/24/2019')-pd.to_datetime('9/30/2019')).days

-98

In [176]:
(pd.to_datetime('5/6/2019')-pd.to_datetime('11/30/2019')).days

-208

In [178]:
96*96350 /208

44469.230769230766

In [11]:
df = pd.read_excel(r'G:\Analytics\Constellation Wine\FY20\Data\Meiomi\Reporting Through 10.13\Meiomie_PGA_SS_9.16_10.13.xlsx',sheet_name='Sheet2')
df['SS_Impressions'] = df['Gross Delivered Impressions']/28
df['SS_Video Completions'] = df['Video Ads 100% Complete']/28
df['SS_Video Views'] = df['SS_Impressions']
df['SS_Video Starts'] = df['SS_Impressions']
df['Publisher'] = 'PGA TOUR'
df['Placement'] ='P13NQ20_0_PGA TOUR_Live Streaming Video_OLVSD_15_CPM_SSPT_DVMM_NAUD_CXL_GEN_PGA Contextual_CRPLAT_2019-09-16_2019-10-31_NA_1 x 1_1x1CC'
    
df = pd.concat([df]*28,ignore_index=False)[['Placement',
                                                       'SS_Impressions',
                                                       'SS_Video Completions',
                                                       'SS_Video Views', 
                                                       'SS_Video Starts', 
                                                       'Publisher']]
df.to_clipboard()

In [21]:
93322/183*99

50485.67213114754

In [24]:
df = pd.read_excel(r'G:\Analytics\Constellation Wine\FY20\Data\KimCrawford\Reporting Through 9.30\Digital\Kim Crawford_Hulu_10.17_Nielsen.xlsx',sheet_name='Sheet2')

In [27]:
df['Imps'] = df['Impressions']/99
pd.concat([df]*99,ignore_index=False).to_clipboard()

In [28]:
df

,Placement Name,Impressions,Imps
0,P10FWM1_0_HULU_Branded Slate - Genre Targeted_...,880253,8891.444444
1,P10BCDY_0_HULU_Video Commercial - Genre Target...,3619869,36564.333333
